In [1]:
import datetime
import pandas as pd
import requests

In [2]:
yesterday = datetime.date.today() - datetime.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format' : 'geojson',
    'starttime' : yesterday - datetime.timedelta(days=26),
    'endtime' : yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
print(response.status_code)

200


In [3]:
# Response of 200 means OK, so we can pull the data out of the result. 
#Since we asked the API for a JSON, we can extract it from the response with the JSON method.

In [4]:
# We need to check the structures of the response data to know where our data is.
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

In [5]:
earthquake_json['metadata']

{'generated': 1731445796000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-10-17&endtime=2024-11-12',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.1',
 'count': 6746}

In [6]:
# Each element in the JSON array features is a row of data for our dataframe.
type(earthquake_json['features'])

list

In [7]:
# Your data will be different depending on the date you run this.
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 4.9,
  'place': '191 km ESE of Ust’-Kamchatsk Staryy, Russia',
  'time': 1731369384328,
  'updated': 1731370895040,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us7000nra5',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us7000nra5&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 369,
  'net': 'us',
  'code': '7000nra5',
  'ids': ',usauto7000nra5,us7000nra5,',
  'sources': ',usauto,us,',
  'types': ',internal-moment-tensor,origin,phase-data,',
  'nst': 86,
  'dmin': 4.818,
  'rms': 1.08,
  'gap': 100,
  'magType': 'mb',
  'type': 'earthquake',
  'title': 'M 4.9 - 191 km ESE of Ust’-Kamchatsk Staryy, Russia'},
 'geometry': {'type': 'Point', 'coordinates': [165.4283, 55.7619, 10]},
 'id': 'us7000nra5'}

In [8]:
# Convert to DataFrame
earthquake_properties_data = [data['properties'] for data in earthquake_json['features']]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,4.90,"191 km ESE of Ust’-Kamchatsk Staryy, Russia",1731369384328,1731370895040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",usauto7000nra5,us7000nra5,",",usauto,us,",",internal-moment-tensor,origin,phase-data,",86.0,4.818000,1.08,100.0,mb,earthquake,"M 4.9 - 191 km ESE of Ust’-Kamchatsk Staryy, R..."
1,0.68,"7 km WNW of Cobb, CA",1731367157080,1731368235199,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc75085486,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",8.0,0.007368,0.01,102.0,md,earthquake,"M 0.7 - 7 km WNW of Cobb, CA"
2,1.90,"20 km W of Point MacKenzie, Alaska",1731366482588,1731366576568,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak024eizkcw2,",",ak,",",origin,phase-data,",NaN,NaN,0.49,NaN,ml,earthquake,"M 1.9 - 20 km W of Point MacKenzie, Alaska"
3,2.15,"4 km W of Magalia, CA",1731366324690,1731366736015,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc75085476,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",6.0,0.082400,0.02,143.0,md,earthquake,"M 2.2 - 4 km W of Magalia, CA"
4,2.00,"33 km WNW of Petersville, Alaska",1731365892605,1731366014224,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak024eiz9nv0,",",ak,",",origin,phase-data,",NaN,NaN,0.38,NaN,ml,earthquake,"M 2.0 - 33 km WNW of Petersville, Alaska"


In [9]:
#  Write Data to CSV :
pd.DataFrame(earthquake_properties_data).to_csv('earthquakes.csv', index=False)